# Tabular data with DataFrames.jl

The `result` that we got above is a `DataFrame` from [DataFrames.jl](http://juliadata.github.io/DataFrames.jl/stable/), one of the major Julia packages for tabular data. 


In [1]:
import Pkg
Pkg.activate(joinpath(@__DIR__, ".."))

  Activating project at `~/documents/presentations/IntroductionToJulia-API-2025`


In [2]:
using DataFrames, CSV

In this notebook, we'll be working on a small dataset from [Kaggle](https://www.kaggle.com/datasets/deepu1109/star-dataset?resource=download), which contains observed features of a few stars. The data is stored in a `CSV` file, which we'll first read into a `DataFrame` object.

In [5]:
filepath = joinpath(@__DIR__, "star_features.csv") # path of data file
df = CSV.read(filepath, DataFrame) # read the file and convert it to a `DataFrame` 

Row,temperature_kelvin,luminosity_Lsun,radius_Rsun,absolute_magnitude,star_type,star_colour,spectral_class
,Int64,Float64,Float64,Float64,Int64,String31,String1
1,3068,0.0024,0.17,16.12,0,Red,M
2,3042,0.0005,0.1542,16.6,0,Red,M
3,2600,0.0003,0.102,18.7,0,Red,M
4,2800,0.0002,0.16,16.65,0,Red,M
5,1939,0.000138,0.103,20.06,0,Red,M
6,2840,0.00065,0.11,16.98,0,Red,M
7,2637,0.00073,0.127,17.22,0,Red,M
8,2600,0.0004,0.096,17.4,0,Red,M
9,2650,0.00069,0.11,17.45,0,Red,M


We can see that this dataframe has 240 rows and 7 columns. To get the names of the columns, you can call the `names` function on the dataframe.

In [6]:
names(df)

7-element Vector{String}:
 "temperature_kelvin"
 "luminosity_Lsun"
 "radius_Rsun"
 "absolute_magnitude"
 "star_type"
 "star_colour"
 "spectral_class"

## Basic stuff
One can do basic selection of these data, for example selecting specific columns, removing columns, adding new columns or rows, etc. In this notebook we won't be using the  `star_type` column, so let's remove it:

In [7]:
select!(df, Not(:star_type))

Row,temperature_kelvin,luminosity_Lsun,radius_Rsun,absolute_magnitude,star_colour,spectral_class
,Int64,Float64,Float64,Float64,String31,String1
1,3068,0.0024,0.17,16.12,Red,M
2,3042,0.0005,0.1542,16.6,Red,M
3,2600,0.0003,0.102,18.7,Red,M
4,2800,0.0002,0.16,16.65,Red,M
5,1939,0.000138,0.103,20.06,Red,M
6,2840,0.00065,0.11,16.98,Red,M
7,2637,0.00073,0.127,17.22,Red,M
8,2600,0.0004,0.096,17.4,Red,M
9,2650,0.00069,0.11,17.45,Red,M


To access specific rows or columns you can access a dataframe like a matrix, e.g. `df[:, 1:3]` gives the first three rows while `df[:, [:a, :b]]` gives the rows selected by name.

Adding new data to a dataframe is also straight forward. One can add new rows or new columns. New rows are added by either providing a vector (where the vector's length matches the number of columns), or by providing a named tuple that explicitly names which column gets which new value.

In [8]:
df1 = df[1:3, 1:3]

Row,temperature_kelvin,luminosity_Lsun,radius_Rsun
,Int64,Float64,Float64
1,3068,0.0024,0.17
2,3042,0.0005,0.1542
3,2600,0.0003,0.102


In [9]:
# add new row by order of columns
push!(df1, [5772, 1, 1])

Row,temperature_kelvin,luminosity_Lsun,radius_Rsun
,Int64,Float64,Float64
1,3068,0.0024,0.17
2,3042,0.0005,0.1542
3,2600,0.0003,0.102
4,5772,1.0,1.0


In [10]:
# add new row by name of columnns
push!(df1, (temperature_kelvin = 3600, radius_Rsun=640, luminosity_Lsun = 65_000))

Row,temperature_kelvin,luminosity_Lsun,radius_Rsun
,Int64,Float64,Float64
1,3068,0.0024,0.17
2,3042,0.0005,0.1542
3,2600,0.0003,0.102
4,5772,1.0,1.0
5,3600,65000.0,640.0


In [12]:
# add new column with given name by field assignment
df1.random_columns = rand(size(df1, 1))
df1

Row,temperature_kelvin,luminosity_Lsun,radius_Rsun,z,random_columns
,Int64,Float64,Float64,Float64,Float64
1,3068,0.0024,0.17,0.328993,0.310709
2,3042,0.0005,0.1542,0.734847,0.990605
3,2600,0.0003,0.102,0.709464,0.405073
4,5772,1.0,1.0,0.583744,0.181268
5,3600,65000.0,640.0,0.425352,0.15829


## Selecting specific rows

There are many ways to select specific rows based on values of certain columns. For example, if we wanted to select the stars with effective temperatures below a certain value, we could do:

In [14]:
df[df.temperature_kelvin .<= 5772, :] # select all the rows for which temperature_kelvin is less than or equal to 5772

Row,temperature_kelvin,luminosity_Lsun,radius_Rsun,absolute_magnitude,star_colour,spectral_class
,Int64,Float64,Float64,Float64,String31,String1
1,3068,0.0024,0.17,16.12,Red,M
2,3042,0.0005,0.1542,16.6,Red,M
3,2600,0.0003,0.102,18.7,Red,M
4,2800,0.0002,0.16,16.65,Red,M
5,1939,0.000138,0.103,20.06,Red,M
6,2840,0.00065,0.11,16.98,Red,M
7,2637,0.00073,0.127,17.22,Red,M
8,2600,0.0004,0.096,17.4,Red,M
9,2650,0.00069,0.11,17.45,Red,M


In that example, we also included all the columns. If we wanted to only get specific columns, we could do

In [16]:
df[df.temperature_kelvin .<= 5772, [:luminosity_Lsun, :radius_Rsun]] # same as above, except we now only select two columns

Row,luminosity_Lsun,radius_Rsun
,Float64,Float64
1,0.0024,0.17
2,0.0005,0.1542
3,0.0003,0.102
4,0.0002,0.16
5,0.000138,0.103
6,0.00065,0.11
7,0.00073,0.127
8,0.0004,0.096
9,0.00069,0.11


We can also combine conditional statements to make more specific selections.

In [20]:
# select rows where temperature <= 5772, and radius is bigger than 0.11 and smaller than 0.67,
# and exclude the `star_colour` column
df[df.temperature_kelvin .<= 5772 .&& (0.11 .< df.radius_Rsun .< 0.67), Not(:star_colour)]

Row,temperature_kelvin,luminosity_Lsun,radius_Rsun,absolute_magnitude,spectral_class
,Int64,Float64,Float64,Float64,String1
1,3068,0.0024,0.17,16.12,M
2,3042,0.0005,0.1542,16.6,M
3,2800,0.0002,0.16,16.65,M
4,2637,0.00073,0.127,17.22,M
5,2700,0.00018,0.13,16.05,M
6,3600,0.0029,0.51,10.69,M
7,3129,0.0122,0.3761,11.79,M
8,3134,0.0004,0.196,13.21,M
9,3628,0.0055,0.393,10.48,M


## Querying and manipulating a `DataFrame`

General querying and data manipulation can be done on `DataFrame`s using standard library functions from DataFrames.jl. However, it is [recommended](https://dataframes.juliadata.org/stable/man/querying_frameworks/#Data-manipulation-frameworks) to use a package dedicated to querying and data manupulation that provides simpler/easier syntax for it. Several options exist, each providing a slightly different variant on the syntax, so you can pick whatever you feel most comfortable in. Here we will use a LINQ-like [Query.jl](https://www.queryverse.org/Query.jl/stable/) syntax, which can be used with any tabular Julia data.

In [21]:
using Query
df

Row,temperature_kelvin,luminosity_Lsun,radius_Rsun,absolute_magnitude,star_colour,spectral_class
,Int64,Float64,Float64,Float64,String31,String1
1,3068,0.0024,0.17,16.12,Red,M
2,3042,0.0005,0.1542,16.6,Red,M
3,2600,0.0003,0.102,18.7,Red,M
4,2800,0.0002,0.16,16.65,Red,M
5,1939,0.000138,0.103,20.06,Red,M
6,2840,0.00065,0.11,16.98,Red,M
7,2637,0.00073,0.127,17.22,Red,M
8,2600,0.0004,0.096,17.4,Red,M
9,2650,0.00069,0.11,17.45,Red,M


Let's perform a basic query that we explain step-by-step:

In [23]:
# `@from ... begin` initializes a query
# here `row` (any name would be fine) is the
# iterable of the tabular data. Think of it as a
# NamedTuple. You can access its values by name with the . syntax
q = @from row in df begin
    # `@where` filters elements where the following
    # expression is true.
    @where row.spectral_class == "O"
    # `@select {stuff... } creates a new named tuple
    @select {row.radius_Rsun, row.absolute_magnitude, Lsquared = row.luminosity_Lsun^2}
    # `@collect` collects the selected results into
    # the specified data structure
    @collect DataFrame
end

Row,radius_Rsun,absolute_magnitude,Lsquared
,Float64,Float64,Float64
1,10.6,-4.7,4.1616e10
2,10.2,-4.4,3.9204e10
3,14.0,-6.23,6.60969e11
4,36.0,-5.76,1.6129e10
5,83.0,-6.89,5.5225e10
6,12.0,-5.91,1.2544e10
7,42.0,-6.12,5.3361e10
8,63.0,-5.83,1.31769e11
9,36.0,-6.34,3.3856e10


As you can see, you don't have to necessarily collect only existing columns, you can even create new ones (specified by name).

Of course, much, much more things are possible to do in such a query. For more, see the documentation of Query.jl.

# Exercises


## Plotting subsets of a dataframe

`DataFrames` has a default dataset that is used in its test suite. Install the `CSV` package, and load this dataset with the command:
```julia
using DataFrames, CSV
iris = DataFrame(CSV.File(
    joinpath(dirname(pathof(DataFrames)), 
    "../docs/src/assets/iris.csv")
))
```

This dataset has various flower species (column `:Species`). For every species create a 1x2 figure with the following plots:

* [1,1] = scatter plot of `SepalLength` vs `SepalWidth`. 
* [1,2] = scatter plot of `PetalLength` vs `PetalWidth`.

for each of these sub-scatter plots, calculate and print the Pearson correlaton coefficient.
